# Script

In [1]:
import numpy as np
import pandas as pd

## ICUp5NewWindow24

In [2]:
df_window = pd.read_csv('./data/output/ICUp5NewWindow24.csv')
df_window.head(3)

,subject_id,day,start_time,end_time,diff_end_start
0,12088486,1,2135-02-07 03:03:00,2135-02-08 03:03:00,24.0
1,12088486,2,2135-02-08 03:03:00,2135-02-09 03:03:00,24.0
2,12088486,3,2135-02-09 03:03:00,2135-02-10 03:03:00,24.0


In [3]:
df_window.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63005 entries, 0 to 63004
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   subject_id      63005 non-null  int64  
 1   day             63005 non-null  int64  
 2   start_time      63005 non-null  object 
 3   end_time        63005 non-null  object 
 4   diff_end_start  63005 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 2.4+ MB


In [4]:
df_window.describe().T

,count,mean,std,min,25%,50%,75%,max
subject_id,63005.0,1.506537e+07,2.875203e+06,10001884.0,12621244.0,15153936.0,17541976.0,19999840.0
day,63005.0,7.655091e+00,8.127624e+00,1.0,3.0,5.0,10.0,100.0
diff_end_start,63005.0,2.337211e+01,3.011440e+00,6.0,24.0,24.0,24.0,29.0


In [5]:
data_types = {
    'subject_id': 'int64',
    'day': 'int64',
    'start_time': 'datetime',
    'end_time': 'datetime'
}

In [6]:
correct_format = '%Y-%m-%d %H:%M:%S'

for col, dtype in data_types.items():
    if dtype == 'datetime':
        df_window[col] = pd.to_datetime(df_window[col], format=correct_format)
    else:
        df_window[col] = df_window[col].astype(dtype)

### A partir desse ponto iremos realizar os procedimentos referentes as extrações específicas

In [7]:
df_window.loc[:, 'temperature'] = np.nan

## ICUp5Temperatura

In [8]:
df_temperatura = pd.read_csv('./data/input/ICUp5Temperatura.csv')
df_temperatura.head(3)

,subject_id,charttime,FahrenheitTemp,CelsiusTemp
0,17123098,2185-07-11 20:29:00.000000 UTC,92.0,33.333333
1,15239071,2153-08-26 21:00:00.000000 UTC,92.0,33.333333
2,10804133,2133-07-08 12:27:00.000000 UTC,92.0,33.333333


In [9]:
df_temperatura.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129989 entries, 0 to 129988
Data columns (total 4 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   subject_id      129989 non-null  int64  
 1   charttime       129989 non-null  object 
 2   FahrenheitTemp  129989 non-null  float64
 3   CelsiusTemp     129989 non-null  float64
dtypes: float64(2), int64(1), object(1)
memory usage: 4.0+ MB


In [10]:
data_types = {
    'subject_id': 'int64',
    'charttime': 'datetime',
    'CelsiusTemp': 'float64',
}

In [11]:
correct_format = '%Y-%m-%d %H:%M:%S.%f UTC'

for col, dtype in data_types.items():
    if dtype == 'datetime':
        df_temperatura[col] = pd.to_datetime(df_temperatura[col], format=correct_format)
    else:
        df_temperatura[col] = df_temperatura[col].astype(dtype)
        
df_temperatura = df_temperatura[data_types.keys()]
        
df_temperatura.head(1)

,subject_id,charttime,CelsiusTemp
0,17123098,2185-07-11 20:29:00,33.333333


In [12]:
df_temperatura.sort_values('CelsiusTemp', ascending=False).head(3)

,subject_id,charttime,CelsiusTemp
27129,15652922,2178-10-06 08:54:00,43.055556
73292,12616482,2150-03-17 09:00:00,41.666667
122279,14418220,2138-07-24 10:37:00,41.277778


In [13]:
for subject_id in df_temperatura.subject_id.unique():
    if df_window[df_window.subject_id == subject_id].shape[0]:
        df_temp = df_window[df_window.subject_id == subject_id]
        df_temp = df_temp[df_temp.end_time.notnull()]
        
        charttime = df_temperatura.query("subject_id == @subject_id")
        for item in charttime:
            for index, row in df_temp.iterrows():
                temperature_on_charttime = [item.CelsiusTemp for _, item in charttime.iterrows() if row.start_time <= item.charttime <= row.end_time]
                
                if not temperature_on_charttime:
                    temperature_on_charttime = np.nan
                else:
                    temperature_on_charttime = max(temperature_on_charttime)
                    
                df_window.loc[index, 'temperature'] = temperature_on_charttime

In [14]:
df_window.sort_values('temperature', ascending=False).head(3)

,subject_id,day,start_time,end_time,diff_end_start,temperature
8359,15652922,17,2178-10-05 09:00:00,2178-10-06 11:18:33,26.0,43.055556
33399,12616482,14,2150-03-16 22:18:00,2150-03-17 16:37:16,18.0,41.666667
34255,14418220,8,2138-07-23 15:02:32,2138-07-24 15:02:32,24.0,41.277778


## ICUpatients5

In [15]:
df_patients = pd.read_csv('./data/input/ICUpatients5.csv')
df_patients.head(3)

,subject_id,hadm_id,stay_id,gender,anchor_age,admittime,intime,outtime,deathtime,intervaloAdmIntUTI,tempoUTI,race,pesoadm,unidpesoadm,altura,unidaltura,imc
0,12088486,29875103,39348271,M,18,2135-02-07 02:57:00 UTC,2135-02-07 03:03:00 UTC,2135-02-12 00:25:00 UTC,2135-02-11 00:01:00 UTC,0,117,UNABLE TO OBTAIN,71.8,kg,178.0,cm,-99.9
1,19850244,27972658,32398411,F,18,2162-06-04 17:38:00 UTC,2162-06-05 15:25:42 UTC,2162-06-09 20:22:03 UTC,NaN,21,100,OTHER,57.7,kg,163.0,cm,-99.9
2,10562205,20149612,34317198,M,18,2149-04-29 23:19:00 UTC,2149-04-30 00:58:00 UTC,2149-05-09 20:54:46 UTC,NaN,1,235,WHITE,98.0,kg,180.0,cm,-99.9


In [16]:
data_types = {
    'subject_id': 'int64',
    'intime': 'datetime',
    'outtime': 'datetime',
}

In [17]:
correct_format = '%Y-%m-%d %H:%M:%S UTC'

for col, dtype in data_types.items():
    if dtype == 'datetime':
        df_patients[col] = pd.to_datetime(df_patients[col], format=correct_format)
    else:
        df_patients[col] = df_patients[col].astype(dtype)
        
df_patients = df_patients[data_types.keys()]

In [18]:
df_patients.loc[:, 'max_temperature_on_day'] = 0
df_patients.loc[:, 'temperature_proportion'] = 0

In [26]:

def get_max_temperature_on_day(row: pd.Series) -> int:
    subject_id = row.subject_id
    
    return df_window[df_window.subject_id == subject_id].temperature.max()

def get_temperature_proportion(row: pd.Series) -> int:
    
    subject_id = row.subject_id
    index = df_window[df_window.subject_id == subject_id].index
    
    days_on_icu = index.shape[0]
    temperature = df_window[df_window.subject_id == subject_id].temperature.dropna()

    if not temperature.shape[0]:
        return np.nan
    
    proportion = temperature.shape[0]/days_on_icu
    
    return proportion

def get_temperature_aggrouped(row: pd.Series) -> int:
    '''
        Calcular a quantidade de janelas de internamento em que ocorre temperatura maior que 37,8. Calcular a proporção em relação a quantidade de dias em internamento.
        ATENÇÃO: Pode haver mais de uma anotação na mesma janela. Considerar a maior temperatura da janela para comparar com o valor citado acima.
        Criar 1 variável contabilizando o percentual de dias de internamento onde houve temperatura elevada:
        - ProporçãoTempElevada: Fórmula: qtde de janelas de 24h com anotação de temperatura elevada / qtde janelas de 24h em internamento
        Categorizar:
        0 - NÃO FOI ENCONTRADA ANOTAÇÃO; 
        1 – FOI ENCONTRADA ANOTAÇÃO, PORÉM NENHUM REGISTRO DE TEMPERATURA ELEVADA; 
        2 - >0 e <= 25%, 
        3 - >25% e <= 50%, 
        4 - >50% e <= 75%, 
        5 - >75% e <= 100%.
    '''
    
    subject_id = row.subject_id
    index = df_window[df_window.subject_id == subject_id].index
    
    temperature = df_window[df_window.subject_id == subject_id].temperature.dropna()

    if not temperature.shape[0]:
        return 0
    
    if temperature.max() <= 37.8:
        return 1
    
    temperature = temperature[temperature > 37.8]
    
    days_on_icu = index.shape[0]
    proportion = temperature.shape[0]/days_on_icu
    
    if proportion > 0 and proportion <= 0.25:
        return 2
    
    if proportion > 0.25 and proportion <= 0.50:
        return 3
    
    if proportion > 0.50 and proportion <= 0.75:
        return 4
    
    if proportion > 0.75 and proportion <= 1:
        return 5
    
    return -1

In [27]:
df_patients.loc[:, 'max_temperature_on_day'] = df_patients.apply(get_max_temperature_on_day, axis=1)

In [28]:
df_patients.loc[:, 'temperature_proportion'] = df_patients.apply(get_temperature_proportion, axis=1)

In [29]:
df_patients.loc[:, 'temperature_aggrouped'] = df_patients.apply(get_temperature_aggrouped, axis=1)

In [30]:
df_patients.temperature_aggrouped.value_counts()

temperature_aggrouped
1    4317
2    1669
3     993
4     358
0      87
5      69
Name: count, dtype: int64

## Finalizando

In [31]:
df_final = df_patients.reset_index()[['subject_id', 'intime', 'outtime', 'max_temperature_on_day', 'temperature_proportion', 'temperature_aggrouped']]
df_final.head()

,subject_id,intime,outtime,max_temperature_on_day,temperature_proportion,temperature_aggrouped
0,12088486,2135-02-07 03:03:00,2135-02-12 00:25:00,37.944444,0.6,2
1,19850244,2162-06-05 15:25:42,2162-06-09 20:22:03,38.500000,1.0,3
2,10562205,2149-04-30 00:58:00,2149-05-09 20:54:46,38.888889,1.0,3
3,17840418,2129-01-14 08:00:00,2129-01-17 16:52:18,38.500000,1.0,2
4,17590005,2134-03-03 03:47:56,2134-03-06 16:58:23,38.333333,1.0,3


In [32]:
df_final.to_csv('./data/output/ICUp5Temperature.csv', index=False)